# inertial frame bulk only

In [39]:
import numpy as np
from mlat.const import dirac


k1 = 1.
k2 = 1.
m1 = 1.
m2 = 1.
qs = np.arange(0, 2 * np.pi, 0.01)

M = np.diag([m1, m1, m2, m2])
iM = np.linalg.inv(M)

i = np.eye(2)
r = dirac(2)
r11 = r.ket(0) * r.bra(0)
r22 = r.ket(1) * r.bra(1)



evals_all = []
evecs_all = []
for q in qs: 
    Q = np.exp(1.j * q)
    # H = np.vstack([
    #     np.hstack([(k1 + k2) * i, -(k1 + k2 * Q.conj()) * i]),
    #     np.hstack([-(k1 + k2 * Q) * i, (k1 + k2) * i]),
    # ])
    H = np.vstack([
        np.hstack([(k1 + k2) * r11 + 1/4 * (k1 + k2) * r22, -(k1 + k2 * Q.conj()) * i]),
        np.hstack([-(k1 + k2 * Q) * i, (k1 + k2) * r11 + 1/4 * (k1 + k2) * r22]),
    ])
    evals, evecs = np.linalg.eig(iM.dot(H))
    idcs = np.argsort(evals)
    evals, evecs = evals[idcs], evecs[idcs]
    evals_all.append(evals)
    evecs_all.append(evecs)
evals_all = np.array(evals_all)
evecs_all = np.array(evecs_all)

# Plot
import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter(x=qs, y=np.sqrt(evals_all[:, i]).real) for i in [0, 1, 2, 3]])
# fig = go.Figure(data=[go.Scatter(x=qs, y=np.sqrt(evals_all[:, i]).real) for i in [0, 1]])
fig.show()

# from plotly.subplots import make_subplots


# fig = make_subplots(rows=len(qs), cols=4)
# for i, q in enumerate(qs):
#     for j in range(4):
#         fig.add_trace(
#             go.Scatter(x=evecs_all[i, j, ::2].real, y=evecs_all[i, j, 1::2].real),
#             row=i+1, col=j+1
#             # title=f"mode: {j}, q: {q}"
#         )
# fig.show()

Exact model

In [14]:
import numpy as np
from mlat.const import dirac


k = 1.
m = 1.
qxs = np.arange(0, 2 * np.pi, 0.1)
qys = np.arange(0, 2 * np.pi, 0.1)
# qxs = np.arange(-np.pi, np.pi, 1)
# qys = np.arange(-np.pi, np.pi, 1)
Qxs = np.exp(1.j * qxs)
Qys = np.exp(1.j * qys)

M = np.diag([m] * 2)
iM = np.linalg.inv(M)

i = np.eye(2)

evals_all = np.empty((len(qys), len(qxs), 2), dtype=np.complex128)
# evecs_all = []
for y, Qy in enumerate(Qys): 
    for x, Qx in enumerate(Qxs):
        Q1 = np.cos(np.pi / 12) * Qx - np.sin(np.pi / 12) * Qy
        Q2 = np.cos(np.pi / 12) * Qx + np.sin(np.pi / 12) * Qy
        H = np.vstack([
            np.hstack([ 2 * k, -k * (1 + Q2.conj())]),
            np.hstack([-k * (1 + Q1), 2 * k]),
        ])
        evals, evecs = np.linalg.eig(iM.dot(H))
        idcs = np.argsort(evals)
        evals, evecs = evals[idcs], evecs[idcs]
        evals_all[y, x] = evals
        # evecs_all.append(evecs)

# Plot
import plotly.graph_objects as go


fig = go.Figure(data=[go.Surface(x=qxs, y=qys, z=np.sqrt(evals_all[:, :, i]).real) for i in [0, 1]])
fig.show()


data = []
for y in range(len(qys)):
    color = '#%02x%02x%02x' % (255 - int(y * 255 / len(qys)), 0, int(y * 255 / len(qys)))
    for i in range(2):
        data.append(go.Scatter(x=qxs, y=evals_all[y, :, i].real, name=f"{qys[y],i}", line=dict(color=color)))

fig = go.Figure(data=data)
fig.show()

# With coriolis force

Using nonlinear solver

In [1]:
import numpy as np
from sympy import symbols, Matrix, nsolve
from sympy.solvers import solve
from sympy.utilities.lambdify import lambdify
from scipy.optimize import fsolve, broyden1

k1 = 1.
k2 = 1.
m = 1.
o = 4

w, = symbols(['w'])

qs = np.arange(0, 2 * np.pi, 0.1)
sols = []
for q in qs:
    Q = np.exp(1.j * q)
    # H = Matrix([
    #     [w**2 * m - (k1 + k2), -2*m*1.j*w*o        , k1 + Q.conj() * k2, 0], 
    #     [0                   , w**2 * m - (k1 + k2), 0                 , k1 + Q.conj() * k2],
    #     [(k1 + Q * k2), 0            , w**2 * m - (k1 + k2), -2*m*1.j*w*o], 
    #     [0            , (k1 + Q * k2), 0                 , w**2 * m - (k1 + k2)]
    # ]) 
    # H = Matrix([
    #     [-w**2 * m + (k1 + k2), 2*m*1.j*w*o        , -k1 - Q.conj() * k2, 0], 
    #     [0                   , -w**2 * m + (k1 + k2), 0                 , -k1 - Q.conj() * k2],
    #     [-(k1 + Q * k2), 0            , -w**2 * m + (k1 + k2), 2*m*1.j*w*o], 
    #     [0            , -(k1 + Q * k2), 0                 , -w**2 * m + (k1 + k2)]
    # ]) 
    # H = Matrix([
    #     [-w**2 * m + (k1 + k2), 2*m*1.j*w*o          , -k1 - Q.conj() * k2, 0], 
    #     [-2*m*1.j*w*o         , -w**2 * m + (k1 + k2), 0                  , -k1 - Q.conj() * k2],
    #     [-(k1 + Q * k2), 0            , -w**2 * m + (k1 + k2), 2*m*1.j*w*o], 
    #     [0            , -(k1 + Q * k2), -2*m*1.j*w*o         , -w**2 * m + (k1 + k2)]
    # ]) 
    H = Matrix([
        [-w**2 * m + (k1 + k2), 2*m*1.j*w*o          , -k1 - Q.conj() * k2, 0], 
        [-2*m*1.j*w*o         , -w**2 * m , 0                  , -k1 - Q.conj() * k2],
        [-(k1 + Q * k2), 0            , -w**2 * m + (k1 + k2), 2*m*1.j*w*o], 
        [0            , -(k1 + Q * k2), -2*m*1.j*w*o         , -w**2 * m ]
    ]) 
    f = lambdify(w, H.det())
    try:
        sol = broyden1(f, [1, 1])
    except:
        sol = [0] * 2
    sols.append(sol)
sols = np.array(sols)

import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Scatter(x=qs, y=sols[:, 0]),
    go.Scatter(x=qs, y=sols[:, 1])
    # go.Scatter(x=qs, y=sols[:, i]) for i in range(1)
])
fig.show()

Single sublattice analytical solution

In [16]:
import numpy as np


k = 1.
m = 1.
w0 = np.sqrt(k / m)
qs = np.arange(0, 2 * np.pi, 0.1)
Qs = np.exp(1.j * qs)

Omega = 3. # Hz

ws = []
for Q in Qs:
    lamb = w0 ** 2 * (Q + Q.conj() - 2)
    w1 = -Omega - np.sqrt(Omega ** 2 - lamb)
    w2 = -Omega + np.sqrt(Omega ** 2 - lamb)
    w3 = Omega - np.sqrt(Omega ** 2 - lamb)
    w4 = Omega + np.sqrt(Omega ** 2 - lamb)

    ws.append([w1, w2, w3, w4])
ws = np.array(ws)


import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Scatter(x=qs, y=ws[:, i].real) for i in range(4)
])
fig.show()

Two sublattice analytical solution

In [13]:
import numpy as np
from sympy import Matrix, conjugate, solve, symbols
from sympy.utilities.lambdify import lambdify

k = 1.
m = 1.
o = 1.

w0 = np.sqrt(k / m)
w, o = symbols(['w', 'o'])

Q, = symbols(['Q'])

# left = Matrix([
#     [w ** 2, 0, 0, 0],
#     [0, w ** 2, 0, 0],
#     [0, 0, w ** 2, 0],
#     [0, 0, 0, w ** 2]
# ])
# right = w0 ** 2 * Matrix([
#     [2, 0, -1 - conjugate(Q), 0],
#     [0, 2, 0, -1 - conjugate(Q)],
#     [-1 - Q, 0, 2, 0],
#     [0, -1 - Q, 0, 2]
# ])
# left = Matrix([
#     [-w ** 2, 2.j * w * o, 0, 0],
#     [-2.j * w * o, -w ** 2, 0, 0],
#     [0, 0, -w ** 2, 2.j * w * o],
#     [0, 0, -2.j * w * o, -w ** 2]
# ])
# right = w0 ** 2 * Matrix([
#     [2, 0, -1 - conjugate(Q), 0],
#     [0, 2, 0, -1 - conjugate(Q)],
#     [-1 - Q, 0, 2, 0],
#     [0, -1 - Q, 0, 2]
# ])

# mat = left + right
# print(mat)
# d = mat.det()
# print(d)
# d = w**8 + 8 * (1 - o**2) * w**6 + \
#     (52 + 16 * o**4 - 32 * o**2 - 2 * Q - 2 * conjugate(Q) * w**4 + (-16 -8 * Q - 8 * conjugate(Q) - 16 * o**2 - 8 * o**2 * conjugate(Q))) * w**2 +\
#     6 + Q**2 + conjugate(Q)**2 - 4 * Q - 4 * conjugate(Q)
d = w**8 - 8 * (1 + o**2) * w**6 + (24 + 16 * o**4 - 2 * (2 + Q + conjugate(Q)) + 32 * o**2) * w**4 +\
    (-32 + 8 * (2 + Q + conjugate(Q)) - 32 * o**2 - 8 * (2 + Q + conjugate(Q)) * o**2) * w**2 +\
    + 16 + (2 + Q + conjugate(Q))**2 - 8 * (2 + Q + conjugate(Q))
print(d)
sol = solve(d, w ** 2)
solf = lambdify([Q, o], sol)

-8*Q + w**8 - w**6*(8*o**2 + 8) + w**4*(-2*Q + 16*o**4 + 32*o**2 - 2*conjugate(Q) + 20) + w**2*(8*Q - o**2*(8*Q + 8*conjugate(Q) + 16) - 32*o**2 + 8*conjugate(Q) - 16) + (Q + conjugate(Q) + 2)**2 - 8*conjugate(Q)


In [37]:
o_ = 1.
sols = []
qs = np.arange(0, 2 * np.pi, 0.1)
for q in qs:
    Q_ = np.exp(1.j * q)
    sols.append(sorted(np.array(solf(Q_, o_))))
sols = np.array(sols)
print(sols.shape)


import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Scatter(x=qs, y=np.sqrt(sols[:, i]).real) for i in range(4)
])
fig.show()

(63, 4)


# With centrifugal force + corioli force

(63, 8)